In [25]:
import os 
import json 
import sagemaker 
from sagemaker.pytorch import PyTorch 
from sagemaker import get_execution_role

sess = sagemaker.Session()
role = get_execution_role()
output_path = "s3://" + sess.default_bucket() + "/DEMO-mnist"

In [37]:
!pygmentize 'code/train.py'

import argparse
import gzip
import json
import logging
import os
import sys

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))

# Based on https://github.com/pytorch/examples/blob/master/mnist/main.py
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
    

In [41]:
local_mode = False
if local_mode:
    instance_type = "local"
else:
    instance_type = "ml.c4.xlarge"
est = PyTorch(
    entry_point="train.py",
    source_dir="code", 
    #source_dir="",
    role=role,
    framework_version="1.5.0",
    py_version="py3",
    instance_type=instance_type,
    instance_count=1,
    volume_size=250,
    output_path=output_path, 
    hyperparameters={"batch-size": 128, "epochs": 1, "learning-rate":1e-3, "log-interval":100},
)

In [24]:
! python train.py --batch-size 100 --epochs 1 --learning-rate 1e-3  --log-interval 100


Traceback (most recent call last):
  File "/home/ec2-user/SageMaker/train.py", line 9, in <module>
    import torch
ModuleNotFoundError: No module named 'torch'


In [39]:
import logging 
import boto3
from botocore.exceptions import ClientError
#Download training and testing data from a public S3 bucket 
def download_from_s3(data_dir="./data", train=True):
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    if train:
        images_file = "train-images-idx3-ubyte.gz"
        labels_file = "train-labels-idx1-ubyte.gz"
    else:
        images_file = "t10k-images-idx3-ubyte.gz"
        labels_file = "t10k-labels-idx1-ubyte.gz"
    
    #download objects
    s3 = boto3.client("s3")
    bucket = f"sagemaker-sample-files"
    for obj in [images_file, labels_file]:
        key = os.path.join("datasets/image/MNIST", obj)
        dest = os.path.join(data_dir, obj)
        if not os.path.exists(dest):
            s3.download_file(bucket, key, dest)
    return

download_from_s3("./data", True)
download_from_s3("./data", False)

In [40]:
prefix = "DEMO-mnist"
bucket = sess.default_bucket()
loc = sess.upload_data(path="./data", bucket=bucket, key_prefix=prefix)
channels = {"training": loc, "testing":loc}

In [42]:
est.fit(inputs=channels)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: pytorch-training-2023-02-01-17-24-47-483


2023-02-01 17:24:47 Starting - Starting the training job...
2023-02-01 17:25:07 Starting - Preparing the instances for training......
2023-02-01 17:26:11 Downloading - Downloading input data...
2023-02-01 17:26:36 Training - Downloading the training image...
2023-02-01 17:27:16 Training - Training image download completed. Training in progress...bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-02-01 17:27:25,521 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-02-01 17:27:25,533 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-02-01 17:27:25,544 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-02-01 17:27:25,546 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-02-01 17:27:25,712 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installe

In [43]:
pt_mnist_model_data = est.model_data
print("Model artifact saved at:\n", pt_mnist_model_data)

Model artifact saved at:
 s3://sagemaker-us-east-2-468328548725/DEMO-mnist/pytorch-training-2023-02-01-17-24-47-483/output/model.tar.gz


In [44]:
%store pt_mnist_model_data

Stored 'pt_mnist_model_data' (str)


In [46]:
!pygmentize code/test_train.py


import json
import os
import sys

import boto3
from train import parse_args, train

dirname = os.path.dirname(os.path.abspath(__file__))

with open(os.path.join(dirname, "config.json"), "r") as f:
    CONFIG = json.load(f)


def download_from_s3(data_dir="/tmp/data", train=True):
    """Download MNIST dataset and convert it to numpy array

    Args:
        data_dir (str): directory to save the data
        train (bool): download training set

    Returns:
        tuple of images and labels as numpy arrays
    """

    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    if train:
        images_file = "train-images-idx3-ubyte.gz"
        labels_file = "train-labels-idx1-ubyte.gz"
    else:
        images_file = "t10k-images-idx3-ubyte.gz"
        labels_file = "t10k-labels-idx1-ubyte.gz"

    # download objects
    s3 = boto3.client("s3")
    bucket = CONFIG["public_bucket"]
    for obj in [images_file, labels_file]:
        key = os.path.join("datasets/image/MNIST", ob